[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ThomasAlbin/Astroniz-YT-Tutorials/blob/main/[ML1]-Asteroid-Spectra/4_spectra_viewer.ipynb)

# Step 4.1: Spectra Viewer

Now that we have parsed the data, let's take a look at some overlapping spectra views of the Bus and Main Group classification schema to get a feeling of the data we are working with.

In [1]:
# Import standard libraries
import os

# Import installed libraries
import ipywidgets
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
# Let's mount the Google Drive, where we store files and models (if applicable, otherwise work
# locally)
try:
    from google.colab import drive
    drive.mount('/gdrive')
    core_path = "/gdrive/MyDrive/Colab/asteroid_taxonomy/"
except ModuleNotFoundError:
    core_path = ""

In [3]:
# Read the data
asteroids_df = pd.read_pickle(os.path.join(core_path, "data/lvl2/", "asteroids.pkl"))

# Plot individual Bus-Class spectra

The following code block allows one to plot Bus-Classes and Main Group spectra, merged into a single figure

In [27]:
# First we set up some nice interactive widgets
top_class_widget = ipywidgets.Dropdown(options = ['Bus_Class', 'Main_Group'])
sub_class_widget = ipywidgets.Dropdown()

# Define a function that updates the content of the sub class based on the top class selection
def update_sub_class(*args):
    sub_class_widget.options = sorted(asteroids_df[top_class_widget.value].unique())
top_class_widget.observe(update_sub_class)

# Set the dark mode and the font size and style
plt.style.use('dark_background')
plt.rc('font', family='serif', size=18)

# Set a function for the (interactive) plots
def plot_single_spec(top_class, sub_class, ylim_fixed=False):
    
    # Create a "wide screen figure"
    plt.figure(figsize=(20,8))

    # Get the number of available spectra. This value is later used to adjust the alpha value ...
    nr_of_spec = float(len(asteroids_df[top_class]==sub_class))

    print(f"Number of ({top_class}) {sub_class} spectra: {nr_of_spec}")
    
    # ... however we do not want to exaggerate it with the transperancy!
    if nr_of_spec > 10:
        nr_of_spec = 10

    asteroids_filtered_df = asteroids_df.loc[asteroids_df[top_class]==sub_class]
        
    # Iterate trough the spectra and plot them
    for _, row in asteroids_filtered_df.iterrows():

        plt.plot(row["SpectrumDF"]["Wavelength_in_microm"],
                 row["SpectrumDF"]["Reflectance_norm550nm"],
                 alpha=1.0/nr_of_spec,
                 color='#ccebc4')

    # Set labels
    plt.xlabel("Wavelength in micrometer")
    plt.ylabel("Reflectance w.r.t. 0.55 micrometer")

    # Set a fixed y limit range if requested
    if ylim_fixed:
        plt.ylim(0.5, 1.5)

    # Properties
    plt.xlim(min(row["SpectrumDF"]["Wavelength_in_microm"]),
             max(row["SpectrumDF"]["Wavelength_in_microm"]))
    plt.grid(linestyle="dashed", alpha=0.3)
    
    plt.show()

# Create an interactive session!
ipywidgets.interactive(plot_single_spec,
                       top_class=top_class_widget,
                       sub_class=sub_class_widget,
                       ylim_fixed=False)

interactive(children=(Dropdown(description='top_class', options=('Bus_Class', 'Main_Group'), value='Bus_Class'…

# Step 4.2: Statistics

How does the classification distribution look like? How are the Bus Classes and corresponding Main Groups distributed? In total we have like 1,400 spectra that shall be used for training, verification and testing. It is likely that we cannot use the Bus Classes, but the Main ones. Otherwise the algorithm might get too biased ...

In [28]:
# Simple description
asteroids_df[["Bus_Class", "Main_Group"]].describe()

,Bus_Class,Main_Group
count,1339,1339
unique,25,4
top,S,S
freq,383,549


In [29]:
# Statistics for the Bus Class
asteroids_df.groupby(["Main_Group", "Bus_Class"])["Bus_Class"].agg(["count"])

count
Main_Group Bus_Class       
C          B             60
           C            141
           Cb            33
           Cg             9
           Cgh           15
           Ch           138
Other      A             16
           D              9
           K             31
           L             34
           Ld            13
           O              1
           R              4
           T             14
           V             35
S          S            383
           Sa            34
           Sk            16
           Sl            49
           Sq            52
           Sr            15
X          X            110
           Xc            60
           Xe            28
           Xk            39

In [30]:
# Statistics for the Main Group
asteroids_df.groupby(['Main_Group'])["Main_Group"].agg(['count'])

,count
Main_Group,
C,396
Other,157
S,549
X,237


# Summary

By applying very simple descriptive statistics we found that the Bus-Class has highly under-represented datasets like e.g., O- or R-Class with 1 and 4 spectra, respectively! Since we want to perform a classification a severe bias in the training, validation and test data should be avoided!

For future considerations we use only the Main Group Classification. However, we keep the Bus Class data, if we want to take a deeper look later, after the classification part.